# Группа ВКонтакте "Хочу в НИУ ВШЭ" 

**А также группы "Хочу в Институт медиа" и "Хочу на факультет права Вышки"**

В этом блокноте я подробно показываю, как импортирую данные для полседующего анализа. В итоге работы я создам тве таблицы: посты из группы *"Хочу в НИУ ВШЭ" и участники групп "Хочу в НИУ ВШЭ", "Хочу в Институт медиа" и "Хочу на факультет права Вышки"*. 

Эти группы я выбрала, так как все три посвящены абитуриентам НИУ ВШЭ. Благодаря анализу постов я хотела выявить некие закономерности. Анализ участников также помог вывести закномерности, посмотреть примерное половое распределение абитуриаентов, предположить некоторые взаимосвязи. О результатах анализа Вы можете узнать из файлов **"pоsts_analys"** и **"people_analys"**. Приятного чтения!

In [9]:
import datetime
import pickle
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

from tqdm import tqdm_notebook

In [10]:
my_user_id = 174249323
version = '5.103' 

with open('ba_hse.txt') as f:
    token = f.read()
token

'vk1.a.McdvROz-3o4AzBwArAUEwwp_P2Jz80gJeUKvbak4v6-RjAYYiL_zzahDwHx2Eg_gDU2mxw_H7sWCMgQRZAS0c3J2zoCuwlm_Jk5YtcjooL4ZEYc1m0E2D3B5YZFfecJTjNWOgW_XagIdy4S3F_IwYACtHxvw_bqP5lfIvGcaTuqy-ZVyaR2iRiUjY2FZhrfE'

In [11]:
def vk_download1(method, parameters):
    url = 'https://api.vk.com/method/' + method + \
'?' + parameters + '&v=' + version + '&access_token=' + token
    
    response = requests.get(url)
    return response.json()['response']

## Загрузка постов из групп ВКонтакте ##

### Группа "Хочу в НИУ ВШЭ" ###

Код выше представляет собой функцию vk_download1, которая используется для загрузки данных с помощью API ВКонтакте (VK).

In [69]:
group_id = '-13515656'

wall = vk_download1('wall.get','owner_id=' + group_id)
wall.keys()

dict_keys(['count', 'items'])

Общий смысл кода выше заключается в том, чтобы получить записи со стены определенной группы или публичной страницы ВКонтакте и изучить структуру ответа API, чтобы понять, какие данные доступны для дальнейшего использования.

In [76]:
wall['count']

TypeError: list indices must be integers or slices, not str

Мы узнаём, что в группе ВКонтакте **"Хочу в НИУ ВШЭ"** 14876 записей.

In [85]:
from tqdm import tqdm_notebook
import time

n = vk_download1('wall.get','owner_id=' + group_id)['count']
n

14876

In [94]:
infa = { 
    'id': [],
    'text': [],
    'likes': [],
    'date': []
}

offset = 0  

while offset < n:
    time.sleep(0.4)
    wall = vk_download1('wall.get', 'owner_id=' + group_id + f'&count=100&offset={offset}')['items']
    
    infa['likes'].extend([item['likes']['count'] for item in wall])
    infa['id'].extend([item['id'] for item in wall])
    infa['text'].extend([item['text'] for item in wall])
    infa['date'].extend([datetime.datetime.fromtimestamp(item['date']) for item in wall])
    
    offset += 100  


Этот код представляет собой метод поэтапной загрузки записей со стены группы или публичной страницы ВКонтакте с целью собрать информацию о каждой записи, включая текст, количество лайков и дату публикации.

In [95]:
infa

{'id': [63043,
  63062,
  63061,
  63059,
  63057,
  63056,
  63054,
  63053,
  63047,
  63042,
  63041,
  63040,
  63039,
  63038,
  63037,
  63035,
  63031,
  63030,
  63029,
  63024,
  63022,
  63019,
  63017,
  63016,
  63015,
  63014,
  63013,
  63010,
  63008,
  63006,
  63005,
  63002,
  63001,
  63000,
  62997,
  62996,
  62992,
  62987,
  62985,
  62983,
  62979,
  62975,
  62974,
  62972,
  62971,
  62969,
  62958,
  62956,
  62954,
  62951,
  62949,
  62947,
  62944,
  62943,
  62942,
  62937,
  62934,
  62933,
  62932,
  62930,
  62929,
  62928,
  62927,
  62926,
  62923,
  62921,
  62920,
  62919,
  62917,
  62915,
  62913,
  62911,
  62907,
  62904,
  62900,
  62899,
  62897,
  62894,
  62891,
  62888,
  62887,
  62886,
  62884,
  62880,
  62879,
  62876,
  62874,
  62871,
  62869,
  62868,
  62865,
  62861,
  62860,
  62846,
  62843,
  62835,
  62830,
  62828,
  62827,
  62824,
  62823,
  62821,
  62818,
  62816,
  62814,
  62813,
  62811,
  62810,
  62808,
  62804,
  62

In [96]:
import pandas as pd
df_posts = pd.DataFrame(infa)
df_posts.head()

,id,text,likes,date
0,63043,17 марта в 12:00 мск представители кампусов НИ...,22,2024-03-07 16:23:42
1,63062,"Конституция, ветви власти, пирамида потребност...",26,2024-03-22 18:49:30
2,63061,7 апреля состоится День открытых дверей бакала...,7,2024-03-21 15:00:04
3,63059,"Делимся дайджестом мероприятий, на которые сто...",9,2024-03-19 13:35:15
4,63057,,16,2024-03-17 12:04:35


In [97]:
df_posts['id'].shape[0]

14876

Так я провела проверку, чтобы убедиться, что выгрузила все записи со страницы сообщества ВКонтакте.

In [98]:
null_counts = df_posts.isnull().sum()
print(null_counts)

id       0
text     0
likes    0
date     0
dtype: int64


Также я убедилась, что в данных нет пропусков.

In [99]:
df_posts.to_csv('posts.csv', index=False)

### Группа "Хочу в Институт медиа" ###

In [100]:
group_id_media = '-184245770'

wall = vk_download1('wall.get','owner_id=' + group_id)
wall.keys()

dict_keys(['count', 'items'])

In [102]:
from tqdm import tqdm_notebook
import time

n = vk_download1('wall.get','owner_id=' + group_id_media)['count']
n

108

Я решила, что данных достаточно мало для анализа, поэтому не стала выгружать их.

### Группа "Хочу на факультет права в Вышке" ###

In [103]:
group_id_law = '-200706217'

wall = vk_download1('wall.get','owner_id=' + group_id)
wall.keys()

dict_keys(['count', 'items'])

In [104]:
from tqdm import tqdm_notebook
import time

n = vk_download1('wall.get','owner_id=' + group_id_law)['count']
n

249

В этом случае мне также показалось, что данных для анализа мало, поэтому выгружать их я не стала. 

**Итогом этого пункта стал датасет "posts", который будет использоваться для анализа постов в группе ВКонтакте "Хочу в НИУ ВШЭ". Этот датасет показывает особенности приёмной компании НИУ ВШЭ.**

## Участники сообществ "Хочу в НИУ ВШЭ", "Хочу в Институт медиа" и "Хочу на факультет права в Вышке" ##

In [20]:
fields = 'bdate, city, home_town, sex'
group_id = '-13515656'

In [26]:
hochu_groups = {
     'Хочу в НИУ ВШЭ' : 'ba_hse',
     'Хочу в Институт медиа' : 'abiturhsemedia',
     'Хочу на факультет права Вышки' : 'pravohse_abituriyent'
    }

In [27]:
def getGroupMembers(group_id):
   
    count = vkDownload('groups.getMembers','group_id=' + group_id)['response']['count']

    n = int(np.ceil(count/1000))  
    
    members = []     
    
    for i in tqdm_notebook(range(n)): 
        current_members = vkDownload('groups.getMembers','group_id='+group_id+'&offset='+str(1000*i))
        members.extend(current_members['response']['items'])
        time.sleep(0.4)
        
    return members

Эта функция предназначена для получения списка участников группы ВКонтакте по её ID. 

In [41]:
hse_people = {}
for group, group_id in hochu_groups.items():
    hse_people[group] = getGroupMembers(group_id)

with open('hse_people', 'wb') as f:
    pickle.dump(hse_people, f)

/var/folders/44/k10r3brn4v9g1my31l1xqgx40000gn/T/ipykernel_1437/1321893740.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(n)):


  0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Этот код создает словарь hse_people, в котором ключами являются названия групп (полученные из словаря hochu_groups), а значениями являются списки участников этих групп, полученные с помощью функции getGroupMembers. Затем этот словарь сохраняется в файл с именем 'hse_people' с использованием модуля pickle. 

Код предназначен для сохранения информации о участниках различных групп ВКонтакте в файле, чтобы ее можно было использовать в дальнейшем анализе или обработке данных.

In [42]:
for group, people in hse_people.items():
    print("Группа: {}, число подписчиков: {}".format(group, len(people)))

Группа: Хочу в НИУ ВШЭ, число подписчиков: 30984
Группа: Хочу в Институт медиа, число подписчиков: 1447
Группа: Хочу на факультет права Вышки, число подписчиков: 1354


Так я смогла узнать число подписчиков в каждой группе.

In [43]:
unique_people = sum(hse_people.values(), [])
unique_people = list(set(unique_people))

len(unique_people)

33021

Всего в трёх сообществах состоят 33021 участник. В анализе я перепроверю информацию и дам более точный ответ.

In [44]:
def getUserInformation():
    
    hse_people_information = {}
    for group, people in hse_people.items():
        current_info = []
        count = len(people)
        n = int(np.ceil(count/500))  
        
        for i in tqdm_notebook(range(n)):
            ids = people[i*500:(i+1)*500]
            ids = ','.join([str(user) for user in ids])
            info = vkDownload('users.get','user_ids={}&fields={}'.format(ids, fields))
            info = info['response']
            current_info.extend(info)
            time.sleep(0.4)
        
        hse_people_information[group] = current_info
    
    return hse_people_information

Эта функция используется для получения информации о пользователях из различных групп ВКонтакте на основе их уникальных идентификаторов.

In [45]:
hse_people_information = getUserInformation()

/var/folders/44/k10r3brn4v9g1my31l1xqgx40000gn/T/ipykernel_1437/3618626029.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(n)):


  0%|          | 0/62 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [46]:
with open('hse_people_information', 'wb') as f:
     pickle.dump(hse_people_information, f)

In [47]:
people_information_df = pd.DataFrame()

for group, people in hse_people_information.items():
    df = pd.DataFrame(people)
    df['city'] = df.city.apply(lambda x: x['title'] if x is not np.nan else np.nan)
    df['group'] = group
    people_information_df = people_information_df.append(df, ignore_index=True)

/var/folders/44/k10r3brn4v9g1my31l1xqgx40000gn/T/ipykernel_1437/3359105334.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  people_information_df = people_information_df.append(df, ignore_index=True)
/var/folders/44/k10r3brn4v9g1my31l1xqgx40000gn/T/ipykernel_1437/3359105334.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  people_information_df = people_information_df.append(df, ignore_index=True)
/var/folders/44/k10r3brn4v9g1my31l1xqgx40000gn/T/ipykernel_1437/3359105334.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  people_information_df = people_information_df.append(df, ignore_index=True)


Этот код создает DataFrame с информацией о пользователях из различных групп ВКонтакте, включая их города проживания и принадлежность к группе, и объединяет эту информацию в один DataFrame.

In [48]:
people_information_df

,id,bdate,city,sex,deactivated,first_name,last_name,can_access_closed,is_closed,home_town,group
0,6892,11.7.2003,Москва,1,banned,Аня,Лазарева,True,False,NaN,Хочу в НИУ ВШЭ
1,8684,17.9.1987,Москва,2,NaN,Андрей,Питушкин,True,False,NaN,Хочу в НИУ ВШЭ
2,13312,NaN,NaN,1,banned,Кира,Щербакова,True,False,NaN,Хочу в НИУ ВШЭ
3,14948,14.6,Praha,2,NaN,Владимир,Пырлик,True,False,NaN,Хочу в НИУ ВШЭ
4,17574,28.2.1986,Москва,1,banned,Майя,Смолицкая,True,False,NaN,Хочу в НИУ ВШЭ
...,...,...,...,...,...,...,...,...,...,...,...
33780,814013443,25.8.2004,Москва,1,NaN,Мадина,Сапаргалиева,True,False,Ташкент,Хочу на факультет права Вышки
33781,816147572,3.2.2000,NaN,1,NaN,Софья,Романова,True,False,,Хочу на факультет права Вышки
33782,830185740,8.8.2006,NaN,1,NaN,Маня,Блин,True,False,,Хочу на факультет права Вышки
33783,830579149,19.10.1994,NaN,1,NaN,Юна,Миллер,True,False,Москва,Хочу на факультет права Вышки


In [56]:
people_information_df.shape

(33785, 11)

Я узнала размеры 'DataFrame".

In [61]:
people_information_df.to_csv('people.csv', index=False)

**Результатом этой части работы стало получение таблицы с информацией об участниках сообществ для абитуриентов НИУ ВШЭ ВКонтакте. Теперь благодаря полученным данным можно проанализировать участников сообществ и узнать особенности абитуриентов НИУ ВШЭ: половозрастной состав, список городов, откуда приехали участники и т.п.**